# and then we go to mario draghi

In [36]:
import pandas as pd
import time
import datetime
import matplotlib
import numpy as np
from statsmodels.distributions.empirical_distribution import ECDF
import math
import requests
from bs4 import BeautifulSoup as bs
import pickle as pk

In [37]:
ad = pd.read_csv('datasets/1976-2020-president.csv')

years = list(set(ad['year']))
states = list(set(ad['state']))
states.sort()
years.sort()

dflist = list()

In [38]:
states.remove('DISTRICT OF COLUMBIA')
years.remove(1976)
years.remove(1980)

#### some function

In [39]:
def foofloat(x):
    try:
        return float(x)
    except:
        return np.NaN

In [40]:
def bls_serial(series_id):
  bsobj = bs(requests.post("https://data.bls.gov/pdq/SurveyOutputServlet", {'series_id': series_id}).text)
  return bsobj.findAll('table')[0].findAll('tr')[2].findAll('td')[0].text.strip().upper()

In [41]:
def ecdf_inv(a, q): return sorted(a)[int(len(a) * q)]
def count_elem(l): return {y:list(l).count(y) for y in set(l)}
def dpc(a,b): return (a/b - 1)*100
def alfa(x): return ''.join([c for c in x if c.isalpha() or c==' ']).strip()

In [42]:
def to_percent(older, col_key, shift=0):
    newer = []
    for y in years:
        for s in states:
            try:
                d = older.loc[y-shift, s, 'DEMOCRAT'][col_key]
                r = older.loc[y-shift, s, 'REPUBLICAN'][col_key]
                newer.append([int(y), s, 100*d/(d+r)]) # % share for each state
            except KeyError as e:
                pass
    return pd.DataFrame(data=newer, columns=['year', 'state', col_key, ])

In [43]:
def to_splatter(a, col_name): # from row-column to table-list
    lst = []
    for y in a.columns[2:]:
        for index, row in a.iterrows():
            lst.append([int(y), row['GeoName'].upper(), row[y], ])
    
    return pd.DataFrame(data=lst, columns=['year', 'state', col_name, ])

In [44]:
def var42(df_res, old_colname, new_colname, jump=1):
    new_df = []
    
    df_res = df_res.set_index(['year', 'state'])
    for y in years:
        for s in states:
            try:
                new_df.append([y,s,] + [df_res.at[(y-i,s), old_colname] for i in range(0, 4, jump)])
            except KeyError:
                pass

    new_df = pd.DataFrame(data=new_df, columns = ['year', 'state',] + [f'{new_colname}_{i}' for i in range(4//jump, 0, -1)])
    return new_df

In [45]:
def beadiff(rowcol, colname, diff = True, shift=1):
    difflist = []

    for s in states:
        tmp = rowcol.loc[(slice(None),s),:]
        yyyy = list({i[0] for i in tmp.index})
        yyyy.sort()
        for y in yyyy[shift:]:
            if diff:
                q = dpc(rowcol.at[(y,s), colname], rowcol.at[(int(y)-shift,s),colname])
            else:
                q = rowcol.at[(y,s),colname]
            difflist.append([y, s, q])
            
    return  pd.DataFrame(data=difflist, columns=['year', 'state', colname])

In [46]:
def whore(df, yy=years, ss=states):
    df = df.set_index(['year','state'])
    x = []
    for s in ss:
        for y in yy:
            try:
                df.loc[(y,s),:]
            except:
                x.append((y,s))
    return x

In [47]:
def sell(df, filename=None):
    df = df.sort_values(by=['year', 'state'])
    dflist.append(df)
    if filename is not None:
        df.to_csv(f'datasets-clean/{filename}.csv', index=False)
    return df

#### y_hat aka presitdent popular vote

In [48]:
pop_vote = pd.read_csv('datasets/1976-2020-president.csv')
pop_vote = pop_vote[['year', 'state', 'party_simplified', 'candidatevotes', ]] # drop useless columns
pop_vote = pop_vote.groupby(['year', 'state', 'party_simplified', ]).sum() # drop multiple candidate
pop_vote = to_percent(pop_vote, 'candidatevotes')
pop_vote = pop_vote.rename(columns={"candidatevotes": "y_votes_percent", })
pop_vote = pop_vote.sort_values(by=['year', 'state'])
pop_vote.to_csv('datasets-clean/popular-vote-y.csv', index=False)
dflist.append(pop_vote)
pop_vote

,year,state,y_votes_percent
0,1984,ALABAMA,38.736605
1,1984,ALASKA,30.944087
2,1984,ARIZONA,32.883272
3,1984,ARKANSAS,38.772412
4,1984,CALIFORNIA,41.775465
...,...,...,...
495,2020,VIRGINIA,55.154687
496,2020,WASHINGTON,59.925503
497,2020,WEST VIRGINIA,30.201468
498,2020,WISCONSIN,50.319063


#### GDP and friends

It was necessary to generate an adjusted series of state GDP because of a change in BEA’s estimation procedure from a Standard Industrial Classification (SIC) basis to a North American Industry Classification System (NAICS) basis in 1997.
Data prior to 1997 were adjusted to avoid any erratic shifts in GDP that year.
While the change to NAICS basis occurred in 1997, BEA also provides estimates under a SIC basis in that year.
~~Our adjustment involved calculating the 1997 ratio of NAICS-based GDP to SIC-based GDP for each state, and multiplying it by SIC-based GDP in all years prior to 1997 to obtain our adjusted series of state-level GDP.~~

In [49]:
gdp_nom_97 = to_splatter(pd.read_csv('datasets/gdp-nominal-63-97.csv'), 'gdp_nom')
gdp_nom_12 = to_splatter(pd.read_csv('datasets/gdp-nominal-97-20.csv'), 'gdp_nom')
gdp_real_97 = to_splatter(pd.read_csv('datasets/gdp-real-77-97-chain-97.csv'), 'gdp_real')
gdp_real_12 = to_splatter(pd.read_csv('datasets/gdp-real-97-20-chain-12.csv'), 'gdp_real')

In [50]:
gdp_nom_97 = gdp_nom_97.set_index(['year', 'state'])
gdp_nom_12 = gdp_nom_12.set_index(['year', 'state'])
gdp_real_97 = gdp_real_97.set_index(['year', 'state'])
gdp_real_12 = gdp_real_12.set_index(['year', 'state'])

In [51]:
# moving to midterms (retrocompatibility)
k = 2
sx = '_mt' if k-1 else ''

#### price indexes

In [52]:
gdp_def_97 = (gdp_nom_97['gdp_nom'] / gdp_real_97['gdp_real']).to_frame('gdp_def').dropna()
gdp_def_12 = (gdp_nom_12['gdp_nom'] / gdp_real_12['gdp_real']).to_frame('gdp_def').dropna()

In [53]:
# price index FAIR
fair_p = pd.concat([beadiff(gdp_def_12,'gdp_def',shift=4), beadiff(gdp_def_97,'gdp_def',shift=4)])

# fix 97jump
tmp = dpc(gdp_def_12.loc[2000], gdp_def_97.loc[1996])
tmp['year'] = 2000
fair_p = pd.concat([fair_p, tmp.reset_index(), ])

# recolumning + appending
fair_p = var42(fair_p, new_colname='fair_p', old_colname='gdp_def', jump=4)

sell(fair_p)

,year,state,fair_p_1
0,1984,ALABAMA,26.788107
1,1984,ALASKA,30.073594
2,1984,ARIZONA,27.619112
3,1984,ARKANSAS,25.124853
4,1984,CALIFORNIA,27.168417
...,...,...,...
495,2020,VIRGINIA,7.549968
496,2020,WASHINGTON,6.646364
497,2020,WEST VIRGINIA,7.411864
498,2020,WISCONSIN,7.602507


In [54]:
# OUR price index
def_index = pd.concat([beadiff(gdp_def_12,'gdp_def', shift=k), beadiff(gdp_def_97,'gdp_def',shift=k)])

if k-1:
    tmp = dpc(gdp_def_12.loc[1996+k], gdp_def_97.loc[1996])
    tmp['year'] = 1996+k
    def_index = pd.concat([def_index, tmp.reset_index(), ])

def_index = var42(def_index, new_colname='def'+sx, old_colname='gdp_def', jump=k)

sell(def_index, 'price-deflator')

,year,state,def_mt_2,def_mt_1
0,1984,ALABAMA,8.923985,16.400541
1,1984,ALASKA,0.219513,29.788690
2,1984,ARIZONA,10.086238,15.926491
3,1984,ARKANSAS,8.706949,15.102903
4,1984,CALIFORNIA,9.427608,16.212370
...,...,...,...,...
495,2020,VIRGINIA,4.216321,3.198776
496,2020,WASHINGTON,3.584248,2.956160
497,2020,WEST VIRGINIA,0.648566,6.719717
498,2020,WISCONSIN,4.323340,3.143273


#### population denom

In [55]:
pop = to_splatter(pd.read_csv('datasets/population-75-21.csv'), col_name='gdp_real')
pop.state = pop.state.apply(alfa)
pop = pop.set_index(['year', 'state'])

#### gdp indexes

In [56]:
# re-generate plain series (see despacito)
gdp_real_plain = gdp_real_12.copy()
for y in range(1996, 1977, -1):
    for s in sorted(states):
        a = gdp_real_97.loc[(y,s),'gdp_real']
        b = gdp_real_97.loc[(y+1,s),'gdp_real']
        c = gdp_real_plain.loc[(y+1,s),'gdp_real']

        gdp_real_plain.at[(y,s),'gdp_real'] = a*b/c

gdp_real_plain = gdp_real_plain.reset_index()

In [57]:
# OUR two-lag + fix 97jump
gdp_real_var = pd.concat([beadiff(gdp_real_97,'gdp_real', shift=k), beadiff(gdp_real_12,'gdp_real', shift=k)])

if k-1:
    tmp = gdp_real_plain.set_index(['year','state'])
    tmp = dpc(tmp.loc[1996+k],tmp.loc[1996])
    tmp['year'] = 1996+k
    gdp_real_var = pd.concat([gdp_real_var, tmp.reset_index(), ])
    
gdp_index = var42(gdp_real_var, new_colname='gdp'+sx, old_colname='gdp_real', jump=k)

sell(gdp_index, 'gdp-growth')

,year,state,gdp_mt_2,gdp_mt_1
0,1984,ALABAMA,9.896919,-0.693998
1,1984,ALASKA,1.171316,18.284477
2,1984,ARIZONA,17.082207,0.363747
3,1984,ARKANSAS,11.296819,0.810684
4,1984,CALIFORNIA,11.894211,3.321546
...,...,...,...,...
495,2020,VIRGINIA,-0.837617,3.881593
496,2020,WASHINGTON,3.341932,12.518139
497,2020,WEST VIRGINIA,-4.653816,4.431850
498,2020,WISCONSIN,-2.571740,2.567721


In [58]:
# TODO pensare a una soluzione
if False:
    gdp_real_12 /= pop.loc[gdp_real_12.index]
    gdp_real_97 /= pop.loc[gdp_real_97.index]

In [59]:
# one-lag FAIR
fair_g = pd.concat([beadiff(gdp_real_97,'gdp_real'), beadiff(gdp_real_12,'gdp_real')])
fair_g = var42(fair_g, new_colname='fair_g', old_colname='gdp_real', jump=4)

sell(fair_g)

,year,state,fair_g_1
0,1984,ALABAMA,4.719412
1,1984,ALASKA,3.995028
2,1984,ARIZONA,10.752575
3,1984,ARKANSAS,7.839078
4,1984,CALIFORNIA,8.028589
...,...,...,...
495,2020,VIRGINIA,-2.757118
496,2020,WASHINGTON,-0.581850
497,2020,WEST VIRGINIA,-3.902932
498,2020,WISCONSIN,-4.006500


#### z growth index

In [60]:
tmp = pd.concat([beadiff(gdp_real_97,'gdp_real'), beadiff(gdp_real_12,'gdp_real')])
quantile = ECDF(pd.read_csv('datasets/us-gdp-quarter-47-21-varpc.csv').GDP_PCH)(3.2)

'''for s in states:
    l = tmp[tmp.state == s].gdp_real
    print(f'{s.ljust(max([len(i) for i in states]))} {ecdf_inv(l, quantile):10.2f}', )'''

GDP_THRESHOLD = sorted([ecdf_inv(tmp[tmp.state == s].gdp_real, quantile) for s in states])[0] if False else 3.2
GDP_THRESHOLD

3.2

In [61]:
zk = 1
zsx = '_mt' if zk-1 else ''
growth_index = pd.concat([beadiff(gdp_real_97,'gdp_real', shift=zk), beadiff(gdp_real_12,'gdp_real', shift=zk)]) ## NON CONVINTO, brick on 97
growth_index.gdp_real = growth_index.gdp_real.apply(lambda x : 1 if x > GDP_THRESHOLD else 0)
growth_index = var42(growth_index, new_colname='z'+zsx, old_colname='gdp_real', jump=zk)

growth_index['z_mt_2'] = (growth_index.z_4 + growth_index.z_3).apply( lambda x: 1 if x else 0)
growth_index['z_mt_1'] = (growth_index.z_2 + growth_index.z_1).apply( lambda x: 1 if x else 0)

sell(growth_index, 'z-growth')

,year,state,z_4,z_3,z_2,z_1,z_mt_2,z_mt_1
0,1984,ALABAMA,1,1,0,0,1,0
1,1984,ALASKA,1,0,0,1,1,1
2,1984,ARIZONA,1,1,0,0,1,0
3,1984,ARKANSAS,1,1,0,1,1,1
4,1984,CALIFORNIA,1,1,0,1,1,1
...,...,...,...,...,...,...,...,...
495,2020,VIRGINIA,0,0,0,0,0,0
496,2020,WASHINGTON,0,1,1,1,1,1
497,2020,WEST VIRGINIA,0,0,0,0,0,0
498,2020,WISCONSIN,0,0,0,0,0,0


#### incumbent

In [62]:
incumbent = pd.read_csv('datasets/incumbent-4president-76-20.csv', sep=';')
tmp = pd.DataFrame(data=[[y,s] for y in years for s in states], columns=['year', 'state'])
incumbent = pd.merge(tmp, incumbent, how='inner', left_on='year', right_on='year')

sell(incumbent, 'incumbent-longrep')

,year,state,incumbent,former_president_again,former_party_morethan_2,sudden_vice,lag_vice
0,1984,ALABAMA,-1,-1,0.0,0,0
1,1984,ALASKA,-1,-1,0.0,0,0
2,1984,ARIZONA,-1,-1,0.0,0,0
3,1984,ARKANSAS,-1,-1,0.0,0,0
4,1984,CALIFORNIA,-1,-1,0.0,0,0
...,...,...,...,...,...,...,...
495,2020,VIRGINIA,-1,-1,0.0,0,1
496,2020,WASHINGTON,-1,-1,0.0,0,1
497,2020,WEST VIRGINIA,-1,-1,0.0,0,1
498,2020,WISCONSIN,-1,-1,0.0,0,1


#### house dummy

In [63]:
house_vote = pd.read_csv('datasets/1976-2020-house-utf8.csv')
house_vote = house_vote[['year', 'state', 'party', 'candidatevotes', ]]
house_vote = house_vote.groupby(['year', 'state', 'party', ]).sum()
house_vote = to_percent(house_vote, 'candidatevotes', shift=2)
house_vote['candidatevotes'] = house_vote['candidatevotes'].apply(lambda x: -1 if x < 50 else 1)
house_vote = house_vote.rename(columns={'candidatevotes':'house_midterm'})
house_vote = pd.concat([house_vote, pd.read_csv('datasets/midterm-fixed.csv')])
house_vote.state = house_vote.state.apply(lambda x : x.upper())

sell(house_vote, 'midterm')

,year,state,house_midterm
0,1984,ALABAMA,1
1,1984,ALASKA,-1
2,1984,ARIZONA,-1
3,1984,ARKANSAS,1
4,1984,CALIFORNIA,1
...,...,...,...
484,2020,VIRGINIA,1
485,2020,WASHINGTON,1
486,2020,WEST VIRGINIA,-1
487,2020,WISCONSIN,1


#### personal income

In [64]:
inc_cap = to_splatter(pd.read_csv('datasets/personal-income-per-capita-72-20.csv'), 'avg_inc')
inc_cap = inc_cap.set_index(['year', 'state'])
#inc_cap.avg_inc = inc_cap.avg_inc.apply(lambda x : math.log(x))
inc_cap = beadiff(inc_cap, 'avg_inc', shift=k)
inc_cap = var42(inc_cap,  'avg_inc', 'avg_inc'+sx, jump=k)

sell(inc_cap, 'income')

,year,state,avg_inc_mt_2,avg_inc_mt_1
0,1984,ALABAMA,16.997412,16.526329
1,1984,ALASKA,1.426071,25.259560
2,1984,ARIZONA,17.224666,15.200083
3,1984,ARKANSAS,16.832890,18.783730
4,1984,CALIFORNIA,15.239614,15.470908
...,...,...,...,...
495,2020,VIRGINIA,8.740216,6.622504
496,2020,WASHINGTON,11.224152,9.655147
497,2020,WEST VIRGINIA,8.665411,10.554562
498,2020,WISCONSIN,8.474146,7.874298


#### unenployment

In [65]:
unn = pd.read_csv('datasets/unemployment-76-21-percent.csv')
furi = 'datasets-clean/serial-id-bsl.pkl'

if False:
    sd = dict()
    for s in set([*unn['Series ID'], *nunn['Series ID']]):
        try:
            sd[s] = bls_serial(s)
        except IndexError:
            sd[s] = f'BADASS_{s}'
    pk.dump(sd, open(furi, 'wb'))
else:
    sd = pk.load(open(furi, 'rb'))

unn = unn.rename(columns={
    "Series ID": "state",
    "Year": "year",
    "Value": "unemp",
    "Period": "month",
    })

# transform
unn.unemp = unn.unemp.apply(foofloat).astype(float)
unn.state = unn.state.map(sd)
unn.month = unn.month.apply(lambda x: x.split('M')[-1])

In [66]:
# replace missing value
glue = pd.read_csv('datasets/unemployment-rate-us-country-47-21.csv')
glue['year'] = glue.DATE.apply(lambda x: x.split('-')[0])
glue['month'] = glue.DATE.apply(lambda x: x.split('-')[1])
glue = glue.set_index(['year','month'])

for i in unn.index:
    unn.at[i, "unemp"] = glue.at[(unn.at[i, "year"], unn.at[i, "month"]), 'UNRATE'] if unn.at[i, "unemp"] == np.NaN else unn.at[i, "unemp"]

In [67]:
# delta%
dmp = unn.set_index(['year','month','state'])

unemp_mt = []
unemp_s = []
for y in years:
    for s in states:
        try:
            m2 = dpc(dmp.at[(y, '10', s), 'unemp'], dmp.at[(y-2, '11', s), 'unemp'])
            m1 = dpc(dmp.at[(y-2, '10', s), 'unemp'], dmp.at[(y-3, '01', s), 'unemp'])
            unemp_mt.append([y,s,m1,m2,])

            unemp_s.append([y, s, dmp.loc[[(y, f'{i:02}', s) for i in range(6,11)], 'unemp'].median(), ])
        except KeyError:
            pass
        
unemp_mt = pd.DataFrame(data=unemp_mt, columns=['year', 'state', 'unemp_mt_1', 'unemp_mt_2'])
unemp_s = pd.DataFrame(data=unemp_s, columns=['year', 'state', 'unemp_rn',])

sell(unemp_s)

,year,state,unemp_rn
0,1984,ALABAMA,11.1
1,1984,ALASKA,9.8
2,1984,ARIZONA,4.5
3,1984,ARKANSAS,8.4
4,1984,CALIFORNIA,7.6
...,...,...,...
495,2020,VIRGINIA,7.0
496,2020,WASHINGTON,9.1
497,2020,WEST VIRGINIA,8.5
498,2020,WISCONSIN,6.1


### merging features

In [68]:
df = pd.DataFrame(data=[[y,s] for y in years for s in states], columns=['year', 'state'])
for d in dflist:
    df = df.merge(d, how='inner', left_on=['year','state'], right_on=['year','state'])

df = df.sort_values(by=['year', 'state'])

df.to_csv('datasets-clean/xxx-2012-insample-final-dataset.csv', index=False)
df[df.year != 2020].to_csv('datasets-clean/xxx-2020-outofsample-final-dataset.csv', index=False)

In [69]:
for d in dflist:
    w = whore(d)
    if len(w):
        print(d.head())
        print('-+'*30)
        print(w)
        print('-+'*30)

In [70]:
0/0
# chernobyl zone

ZeroDivisionError: division by zero

exp fail

In [ ]:
tmp = []
for y in years:
    for s in states:
        m2 = unn[unn.year >= y-1][unn.year <= y][unn.state == s].unemp.median()
        m1 = unn[unn.year >= y-3][unn.year <= y-2][unn.state == s].unemp.median()
        tmp.append([y,s,m1,m2])
mid_unem = pd.DataFrame(data=tmp, columns=['year', 'state', 'unemp_12', 'unemp_34'])
dflist.append(mid_unem)
mid_unem

In [ ]:
unn = var42(unn, 'unemp', 'unemployment-index', 'unemp')
dflist.append(unn)
unn

In [ ]:
gdp_index['gdp_34'] = (gdp_index['gdp_3']+gdp_index['gdp_3'])/2
gdp_index['gdp_12'] = (gdp_index['gdp_1']+gdp_index['gdp_2'])/2

del gdp_index['gdp_1']
del gdp_index['gdp_2']
del gdp_index['gdp_3']
del gdp_index['gdp_4']

In [ ]:
gdp_boot = gdp_real.copy()
gdp_exp = gdp_boot.set_index(['year', 'state'])
for s in states:
    print(gdp_exp.loc[(slice(None),s),:].gdp_real.apply(lambda x : math.exp((x - gdp_exp.gdp_real.mean())/ math.sqrt(gdp_exp.gdp_real.var()))))
    break

us stuff

In [ ]:
gdp_us = pd.read_csv('datasets/gdp-nomina-47-20-chain-12.csv')
gdp_us.DATE = gdp_us.DATE.apply(lambda x: int(str(x).split('-')[0]))
s = gdp_us[gdp_us.DATE > 1975].GDPC1_PC1
s.hist()

In [ ]:
ecdf = ECDF(s)
ecdf(2.8)

qnt = 0.67
std_err = np.sqrt(s.var())/2
ecdf_inv(s, qnt), len(s)*(1-qnt), std_err

#### house

In [ ]:
'''
house_vote = pd.read_csv('datasets/1976-2020-house-utf8.csv')
house_vote = house_vote[['year', 'state', 'party', 'candidatevotes', ]]
house_vote = house_vote.groupby(['year', 'state', 'party', ]).sum()
house_vote = to_percent(house_vote, 'candidatevotes')
house_vote = house_vote.rename(columns={"candidatevotes": "houserep_votes_percent", })
#'''

#### gasoline and friends + approval

In [ ]:
gas = pd.read_csv('datasets/gasoline-93-21.csv')
gas.date = gas.date.apply(lambda x: time.mktime(datetime.datetime.strptime(x,"%m/%d/%Y").timetuple()))
gas.date = gas.date.apply(lambda x: datetime.datetime.fromtimestamp(int(x)))
gas

#### houses prices and rent + personal income

In [ ]:
f = open('datasets/house-chain-00.csv', 'rt')
lines = f.readlines()
houses = [[i.strip() for i in l.split(sep = '$') ] for l in lines ]
houses = pd.DataFrame(data=houses, columns=['state', 2000, 1990, 1980, 1970, 1960, 1950, 1940 ])
houses.head()